In [443]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as pyo
from plotly.subplots import make_subplots

In [444]:
url = "https://ex.login.net.vn/check/"
# code = "D6037253"
code = "D6082563"

In [445]:
df = pd.read_html(url + code)[1]
df.head(10)

,#,TIME,ONLINE TIME,DOWNLOAD,UPLOAD,MAC
0,1,18/04/2023 12:11:44,00:29:14,0.65 MB,0.29 MB,A8:9C:ED:37:54:E3
1,2,18/04/2023 10:06:27,02:29:46,707.51 MB,55.48 MB,98:22:EF:53:81:45
2,3,18/04/2023 00:54:11,10:48:04,617.57 MB,34.1 MB,A8:9C:ED:37:54:E3
3,4,17/04/2023 21:23:33,03:32:01,427.42 MB,69.48 MB,98:22:EF:53:81:45
4,5,16/04/2023 20:43:18,24:33:50,2.48 GB,160.67 MB,98:22:EF:53:81:45
5,6,16/04/2023 06:55:35,00:08:22,32.59 MB,3.49 MB,98:22:EF:53:81:45
6,7,15/04/2023 19:01:11,11:49:49,1.55 GB,104.71 MB,98:22:EF:53:81:45
7,8,15/04/2023 09:41:39,02:40:51,2.42 GB,41.28 MB,98:22:EF:53:81:45
8,9,14/04/2023 17:47:00,08:06:13,4.32 GB,218.97 MB,98:22:EF:53:81:45
9,10,14/04/2023 09:55:27,00:50:13,34.61 MB,5.45 MB,98:22:EF:53:81:45


In [446]:
df = df.replace({
    "98:22:EF:53:81:45": "Laptop",
    "A8:9C:ED:37:54:E3": "Phone"
})

In [447]:
df = df.drop(columns = ["#"])

df["TIME"] = pd.to_datetime(df["TIME"], format="%d/%m/%Y %H:%M:%S")

df["DATE"] = df["TIME"].dt.date

df["ONLINE TIME"] = df["ONLINE TIME"].apply(lambda x : int(x[0:2]) + int(x[3:5])/60)

df["DOWNLOAD"] = df["DOWNLOAD"].str.split(" ").str[0].astype(float) * df["DOWNLOAD"].str.split(" ").str[1].replace({"MB": 1,"GB": 1000})

df["UPLOAD"] = df["UPLOAD"].str.split(" ").str[0].astype(float) * df["UPLOAD"].str.split(" ").str[1].replace({"MB": 1,"GB": 1000})

df["SUM"] = df["DOWNLOAD"] + df["UPLOAD"]

In [448]:
min_valid_date = df["DATE"].min()
max_valid_date = df["DATE"].max()

In [449]:
df1 = pd.DataFrame({
    "DATE":  np.tile(pd.date_range(df["DATE"].min(), df["DATE"].min() + pd.Timedelta(days = 20), freq = "D").date, 2),
    "MAC": [df["MAC"].unique()[0] for i in range(21)] + [df["MAC"].unique()[1] for i in range(21)],
})

df = pd.concat([df, df1]).reset_index(drop = True)
df = df.drop_duplicates(subset = ["SUM", "DATE", "MAC"]).reset_index(drop = True)

df["SUM"] = df[df["DATE"].between(min_valid_date, max_valid_date)]["SUM"].fillna(0)

In [450]:
data = df.groupby(["DATE", "MAC"]).agg({"ONLINE TIME": "sum", "SUM": "sum", "UPLOAD": "sum"}).reset_index()

fig1 = px.bar(
    data, x = "DATE", y = "SUM", color = "MAC", barmode = "stack", title = "Dung lượng theo ngày",
    labels = {"DATE": "Ngày", "SUM": "MB"}
)

In [451]:
data = df.groupby("DATE")[["SUM"]].sum(min_count=1).sort_index(ascending = True).cumsum(skipna = False).reset_index()

fig2 = px.line(
    data, x = "DATE", y = "SUM",
    color_discrete_sequence = ["green"], title = "Tổng dung lượng qua ngày",
    labels = {"DATE": "Ngày", "SUM": "MB"}
)

fig2.add_scatter(
    x = [data["DATE"].min(), data["DATE"].max()], y = [3000, 60000], 
    name = "Limit", mode = "lines", showlegend = False, line = {"color": "lightgray"}
)

In [454]:
data

[60.84, -0.8400000000000034]

In [455]:
data = [min(round(df["SUM"].sum() / 1000, 2), 60)]
if data[0] < 60:
    data.append(60 - data[0])

fig3 = px.pie(
    values = data,
)

fig3.update_traces(
    textinfo='value', 
    marker = dict(colors = ["green", "lightgray"]),
)

In [453]:
combine_fig = make_subplots(
    rows = 2, cols = 3, shared_xaxes = "all",
    specs = [[{"colspan" : 2}, {}, {}], [{"colspan" : 2}, {}, {"type": "domain"}]],
    column_widths=[0.5, 0.3, 0.2],
    subplot_titles = ("Dung lượng theo ngày", "", "", "Tổng dung lượng qua ngày")
)

combine_fig.add_trace(fig1['data'][0], row = 1, col = 1)
combine_fig.add_trace(fig1['data'][1], row = 1, col = 1)
combine_fig.add_trace(fig2['data'][0], row = 2, col = 1)
combine_fig.add_trace(fig2['data'][1], row = 2, col = 1)
combine_fig.add_trace(fig3['data'][0], row = 2, col = 3)

combine_fig.update_layout(barmode = "stack")

pyo.plot(combine_fig, filename = "report.html", auto_open = False)

'report.html'